In [1]:
import pandas as pd
import os

def get_all_categories(data_directory):
    # List all directories (which represent categories) in the data folder
    categories = [folder for folder in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, folder))]
    return categories

# Path to the dataset
data_directory = 'data/Train_Full'

# Get the list of all categories (topics)
categories = get_all_categories(data_directory)

# Print the categories
print("List of all categories (topics):")
print(categories)

List of all categories (topics):
['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh', 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa', 'Vi tinh']


In [2]:
def load_vntc_data_with_labels(data_directory):
    texts = []
    labels = []

    categories = get_all_categories(data_directory)  # Get the list of all categories

    for category in categories:
        category_path = os.path.join(data_directory, category)

        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            with open(file_path, 'r', encoding='utf-16') as file:
                content = file.read().strip()
                texts.append(content)
                labels.append(category)  # Append category as label

    return pd.DataFrame({'Labels': labels, 'Text': texts})

# Load the data with labels
data_train_directory = 'data/Train_Full'
data_test_directory = 'data/Test_Full'
data_train = load_vntc_data_with_labels(data_train_directory)
data_test = load_vntc_data_with_labels(data_test_directory)


In [3]:
data_train

,Labels,Text
0,Chinh tri Xa hoi,Thành lập dự án POLICY phòng chống HIV/AIDS ở ...
1,Chinh tri Xa hoi,Hơn 16.000 khách đến vịnh Nha Trang Theo trực ...
2,Chinh tri Xa hoi,TPHCM: Khai trương dịch vụ lặn biển săn cá mập...
3,Chinh tri Xa hoi,Du lịch VN sẽ có tư vấn nước ngoài Ông Phạm Từ...
4,Chinh tri Xa hoi,Quy chế tuyển sinh 2006: Không làm tròn điểm t...
...,...,...
33754,Vi tinh,Điện thoại di động tương lai trông như thế nào...
33755,Vi tinh,Internet sẽ tăng tốc 1.000 lần \nTrong tương l...
33756,Vi tinh,Phần lớn thế giới thứ 3 thất bại với chính phủ...
33757,Vi tinh,'Doom 3' giành chiến thắng kép \nTrò chơi hành...


In [4]:
data_test

,Labels,Text
0,Chinh tri Xa hoi,Mạo hiểm rừng Đa Mi Cuộc hành quân khám phá th...
1,Chinh tri Xa hoi,Tàu du lịch cao tốc Cần Thơ - Phnom Penh Công ...
2,Chinh tri Xa hoi,Miền Trung được mùa khách Thái Đoàn du khách T...
3,Chinh tri Xa hoi,7 kỳ quan mới của thế giới Cầu Akashi - Kaikyo...
4,Chinh tri Xa hoi,Khối A thi được mấy trường? Thi khối A vào ĐH ...
...,...,...
50368,Vi tinh,Phần mềm chống tải nhạc bất hợp pháp\nNhóm ngh...
50369,Vi tinh,Linux tăng ảnh hưởng trên thị trường máy chủ\n...
50370,Vi tinh,Napster phát không máy nghe nhạc\nCông ty phát...
50371,Vi tinh,Intel giới thiệu chip và chipset chuyên biệt h...


In [5]:
data_test['Labels'].value_counts()

Labels
Chinh tri Xa hoi    7567
The gioi            6716
The thao            6667
Van hoa             6250
Suc khoe            5417
Kinh doanh          5276
Vi tinh             4560
Phap luat           3788
Khoa hoc            2096
Doi song            2036
Name: count, dtype: int64

In [ ]:
import re
from underthesea import text_normalize, word_tokenize

class Preprocessing:
    def __init__(self, file_path):
        self.file_path = file_path
        self.stopwords = self.read_stopwords(file_path)

    def read_stopwords(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().splitlines())
        return stopwords
    
    def preprocessing_text(self, text):
        text = text.lower().strip()

        text = re.sub(r'\b(\d+)k\b', r'\1 ngàn', text)
        text = re.sub(r'\b(\d+)%\b', r'\1 phần trăm', text)
        text = re.sub(r'\b(\d+)m\b', r'\1 mét', text)
        text = re.sub(r'\b(\d+)s\b', r'\1 giây', text)
        text = re.sub(r"\b(\d+)'\b", r'\1 phút', text)
        text = re.sub(r'\b(\d+)h\b', r'\1 giờ', text)

        text = re.sub(r'[^0-9a-zA-ZÀ-ỹ\s]', '', text)

        text = text_normalize(text)
        words = word_tokenize(text, format="text").split()
        
        words = [w for w in words if w not in self.stopwords]

        return " ".join(words)

In [7]:
data_train = data_train.sample(n=500).reset_index(drop=True)
data_test = data_test.sample(n=300).reset_index(drop=True)

In [8]:
data_train

,Labels,Text
0,Suc khoe,"Mùa hè ăn gì để đẹp da?\nTheo Đông y, về mùa h..."
1,Phap luat,Án mạng nghiêm trọng từ chuyện nợ nần\nTối mùn...
2,Vi tinh,ADSL chinh phục người dùng bằng tốc độ\nVới tố...
3,Phap luat,Hà Kiều Anh vẫn yêu cầu đòi bồi thường 20.000 ...
4,The gioi,"Hôm 9/3, Trung Quốc đã trả đũa các chỉ trích n..."
...,...,...
495,The thao,Ronaldo tiếp tục lập công để dập tắt mọi mối n...
496,The thao,Lehmann đưa Arsenal vào chung kết Thủ thành Le...
497,The thao,Tuyển đại lý cầu thủ của FIFA: hai thí sinh đề...
498,Chinh tri Xa hoi,Như chỗ không người “Tè đường” trở thành căn b...


In [9]:
file_path='vietnamese-stopwords-dash.txt'
preprocessor = Preprocessing(file_path)
data_train['Text_Processing'] = data_train['Text'].apply(preprocessor.preprocessing_text)
data_test['Text_Processing'] = data_test['Text'].apply(preprocessor.preprocessing_text)

In [10]:
data_train

,Labels,Text,Text_Processing
0,Suc khoe,"Mùa hè ăn gì để đẹp da?\nTheo Đông y, về mùa h...",mùa hè đẹp da đông_y mùa hè thức_ăn_nhiệt rôm ...
1,Phap luat,Án mạng nghiêm trọng từ chuyện nợ nần\nTối mùn...,án_mạng nghiêm_trọng nợ_nần tối mùng 5 tết 261...
2,Vi tinh,ADSL chinh phục người dùng bằng tốc độ\nVới tố...,adsl chinh_phục tốc_độ tốc_độ đường truyền gấp...
3,Phap luat,Hà Kiều Anh vẫn yêu cầu đòi bồi thường 20.000 ...,hà_kiều đòi bồi_thường 20000 usd trưa tand tỉn...
4,The gioi,"Hôm 9/3, Trung Quốc đã trả đũa các chỉ trích n...",hôm 93 trung_quốc trả_đũa chỉ_trích nhân_quyền...
...,...,...,...
495,The thao,Ronaldo tiếp tục lập công để dập tắt mọi mối n...,ronaldo lập_công dập tắt nghi_ngờ phong_độ đội...
496,The thao,Lehmann đưa Arsenal vào chung kết Thủ thành Le...,lehmann arsenal chung_kết_thủ thành lehmann đồ...
497,The thao,Tuyển đại lý cầu thủ của FIFA: hai thí sinh đề...,tuyển đại_lý cầu_thủ fifa hai thí_sinh thất_bạ...
498,Chinh tri Xa hoi,Như chỗ không người “Tè đường” trở thành căn b...,chỗ tè đường căn_bệnh truyền_nhiễm văn_minh ph...


In [11]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
from underthesea import text_normalize, word_tokenize

In [12]:
label_encoder = LabelEncoder()
data_train['Labels'] = label_encoder.fit_transform(data_train['Labels'])
data_test['Labels'] = label_encoder.transform(data_test['Labels'])

In [ ]:
from collections import Counter

def tokenizer_fn(s: str):
    return word_tokenize(s, format="text").split()

def build_vocab_from_texts(text_series, min_freq=1, specials=("<unk>", "<pad>")):
    counter = Counter()
    for s in text_series:
        counter.update(tokenizer_fn(s))
    itos = list(specials)  # index -> token
    for tok, freq in counter.most_common():
        if freq >= min_freq and tok not in specials:
            itos.append(tok)
    stoi = {tok: i for i, tok in enumerate(itos)}  # token -> index
    return stoi, itos

stoi, itos = build_vocab_from_texts(data_train['Text_Processing'], min_freq=1)
UNK_IDX = stoi["<unk>"]
PAD_IDX = stoi["<pad>"]

def text_pipeline(x: str):
    return [stoi.get(tok, UNK_IDX) for tok in tokenizer_fn(x)]

def label_pipeline(y):
    return int(y)


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from itertools import chain
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def texts_to_ids(text_series):
    seqs = [text_pipeline(s) for s in text_series]
    flat = list(chain.from_iterable(seqs))
    return flat

corpus_ids = texts_to_ids(data_train['Text_Processing'])

class NextTokenDataset(Dataset):
    def __init__(self, token_ids, seq_len=50, step=1):
        self.token_ids = token_ids
        self.seq_len = seq_len
        self.step = step
        self.n_samples = max(0, (len(token_ids) - 1 - seq_len) // step + 1)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        i = idx * self.step
        x = torch.tensor(self.token_ids[i:i+self.seq_len], dtype=torch.long)
        y = torch.tensor(self.token_ids[i+1:i+self.seq_len+1], dtype=torch.long)
        return x, y

SEQ_LEN = 50
BATCH_SIZE = 64
dataset = NextTokenDataset(corpus_ids, seq_len=SEQ_LEN, step=1)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
len_vocab = len(itos)
len_vocab, len(dataset)


(17819, 86835)

In [ ]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, hidden_dim=512, num_layers=2, dropout=0.2, pad_idx=PAD_IDX):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True
        )
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x, hidden=None):
        # x: [B, T]
        emb = self.emb(x)           # [B, T, E]
        out, hidden = self.lstm(emb, hidden)  # out: [B, T, H]
        out = self.drop(out)
        logits = self.fc(out)       # [B, T, V]
        return logits, hidden

model = LSTMLanguageModel(vocab_size=len_vocab, emb_dim=256, hidden_dim=512, num_layers=2, dropout=0.3).to(device)


In [ ]:
import math
from torch.nn.utils import clip_grad_norm_

EPOCHS = 5
LR = 2e-3
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

def train_one_epoch(model, loader, optimizer, criterion, clip=1.0):
    model.train()
    total_loss, total_tokens = 0.0, 0
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)  # [B, T]

        optimizer.zero_grad()
        logits, _ = model(x)  # [B, T, V]
        loss = criterion(logits.reshape(-1, logits.size(-1)), y.reshape(-1))
        loss.backward()
        clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        with torch.no_grad():
            tokens = (y != PAD_IDX).sum().item()
            total_loss += loss.item() * tokens
            total_tokens += tokens

    ppl = math.exp(total_loss / max(1, total_tokens))
    return total_loss / max(1, total_tokens), ppl

for epoch in range(1, EPOCHS+1):
    avg_loss, ppl = train_one_epoch(model, loader, optimizer, criterion, clip=1.0)
    print(f"Epoch {epoch:02d} | loss/token = {avg_loss:.4f} | ppl = {ppl:.2f}")


KeyboardInterrupt: 

In [ ]:
import torch.nn.functional as F

def top_k_filtering(logits, top_k=0):
    if top_k > 0:
        values, _ = torch.topk(logits, top_k)
        min_values = values[:, -1].unsqueeze(1)
        logits = torch.where(logits < min_values, torch.full_like(logits, float('-inf')), logits)
    return logits

def generate_text(
    prompt: str,
    max_new_tokens: int = 80,
    temperature: float = 1.0,
    top_k: int = 0
):
    model.eval()
    ids = [stoi.get(tok, UNK_IDX) for tok in tokenizer_fn(prompt)]
    x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)  # [1, T]
    hidden = None
    generated = ids[:]

    with torch.no_grad():
        for _ in range(max_new_tokens):
            logits, hidden = model(x[:, -SEQ_LEN:], hidden)  
            next_logits = logits[:, -1, :]  # [1, V]
            next_logits = next_logits / max(1e-6, temperature)
            next_logits = top_k_filtering(next_logits, top_k=top_k)
            probs = F.softmax(next_logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1).item()
            generated.append(next_id)
            x = torch.tensor([generated], dtype=torch.long, device=device)

    return " ".join(itos[idx] for idx in generated)

seed = "sữa chua uống men sống nuvi là sản phẩm"
print(generate_text(seed, max_new_tokens=40, temperature=0.9, top_k=50))
